In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

In [2]:
url = "https://store.steampowered.com/games/"
response = requests.request("GET",url)
data = BeautifulSoup(response.text,'html.parser')

In [3]:
## Get the links of top 5 games
links = []
for link in data.find_all("a",{"class" : "tab_item"},limit=5):
  links.append(link.get('href'))

In [4]:
s=[]
R=[]
df=[]
#iterating through each link
for link in links:
    url = link
    response = requests.request("GET",url)
    data = BeautifulSoup(response.text,'html.parser')
    #get the title of the game
    title = data.find("div",{"id":"appHubAppName"}).text 
    #get count of positive user reviews
    try:
        m = data.find('span',{'class':'nonresponsive_hidden responsive_reviewdesc'}).text.split(' ')
        Rating = math.ceil(((int(m[1].replace('%','')))/100)*int(m[4])) 
    except:
        Rating = "Insufficient User Reviews"
    #get developer and publisher name
    developer = data.find('div',{"id":"developers_list"})
    publisher = data.find_all('div',{"class":"summary column"})[2].text.replace('\n','')
    #get system requirements
    for li in data.find('div', {"class":['game_area_sys_req_full', 'game_area_sys_req_leftCol']}).find('ul', class_='bb_ul').find_all('li'):
        s.append(li.text)
    #get top 10 reviews
    str="https://steamcommunity.com/app/" + link.split("/")[4] + "/reviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=all"
    url = str
    response = requests.request("GET",url)
    data = BeautifulSoup(response.text,'html.parser')
    for i in data.find_all("div",{"class":"apphub_CardTextContent"},limit=10):
        R.append(((("".join(i.text.split("\t"))).strip()).splitlines())[-1])
    #add all the details to dataframe
    df.append({'Title': title,
                'Positive Review Count' : Rating,
                'Developer' : developer.text.strip(),
                'Publisher' : publisher,
                'System Requirements' : "🔴".join(s[:]),
                'Review' : "🔴".join(R[:])
                   })
    
    s.clear()
    R.clear()
df1 = pd.DataFrame(df)
df = df1.style.set_table_styles([dict(selector='th', props=[('text-align', 'centre')])])
df.set_properties(**{'text-align': 'centre'}).hide_index()



    

In [5]:
df1.to_csv('C:/Users/Shishir/Desktop/File.csv', mode='a', index=False, header=True)
